# Hands-on Workshop: AI Agents with Tool Use using Gemini

Welcome to this hands-on workshop! In this notebook, we will explore how to build an AI Agent capable of using tools (Function Calling) using Google's Gemini API.

# Introduction:
In the context of agentic AI, tools are external capabilities the LLM can invoke, for example:
*   APIs
*   Database queries
*   Internal services
*   Third-party systems
*   Internal functions written in code

They turn the LLM from something that just talks into something that can act.

Remember, LLMs on their own are stateless, have no access to real-time systems, and can’t take action.

# But Give Them Tools, and They Can:
*  Fetch data from your internal systems
*  Trigger events (e.g., send an email, create a JIRA ticket)
*  Access structured data like calendars, dashboards, or CRMs
*  Run pre-written logic based on business rules

*This is how generation turns into execution.*

## What we will build
We will create a simple agent that can:
1. Answer general questions.
2. Perform mathematical calculations using a defined tool.
3. Retrieve "live" information (mocked) using a defined tool.

# Why Tools Matter

1.   **They unlock execution**: Without tools, your agent is just an assistant that makes suggestions. With tools, it can complete $workflows^*$ end-to-end.
2.   **They increase precision**: Rather than hallucinating, the LLM can ask the right system directly — “What’s the actual order status?” instead of making up a delay reason.
3. **They let you control risk**: You define what’s exposed. The LLM can’t do anything outside of the tools you register.
4. **They enable composability**: If you want to combine your CRM, calendar, and email stack into one assistant, you can expose each of those as tools and let the LLM orchestrate them.

## Prerequisites
You need a Google AI Studio API Key. If you don't have one, get it here: https://aistudio.google.com/app/apikey

Make sure you add it in the colab secrets manager.


* A workflow is a recipe an agent uses for task completion. It is a structured, predefined sequence of multiple steps and uses tools to take decisions autonomously. Example: Travel booking, data extraction, sending birthday emails. ***n8n*** is the platform that hosts and runs workflows.

What is Agentic AI, and how can we use it?

Most AI we encounter today is what we call "passive" or "reactive" AI. It responds to our prompts and requests. Think of asking your smart speaker to play a song or a chatbot answering your customer service questions. They’re helpful, sure, but they’re fundamentally waiting for us to tell them what to do.

Agentic AI, on the other hand, takes a fundamentally different approach. It’s all about giving AI the ability to proactively pursue goals, make decisions, and take actions in the world, even without explicit instructions every step of the way.

## 0. Setup and Configuration
First, let's install and import the library and configure your API key.

Recommendation: Please use the Colab Secrets Manager

[Student Hit with a 55,444.78$ bill for leaking Gemini API Key](https://www.reddit.com/r/googlecloud/comments/1noctxi/student_hit_with_a_5544478_google_cloud_bill/)

In [ ]:
# Install the necessary library
!pip install google-ai-generativelanguage

# Import the Libraries and Generate a Client Object

The Client Object acts as a single entrypoint for various API services, promoting consistency and simplifying credential and configuration management across different API calls.

In [ ]:
import google.generativeai as genai
from google.colab import userdata
# Get API key from Colab secrets manager
GOOGLE_API_KEY = userdata.get('gkey')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemma-3-27b-it')
response = model.generate_content(
    'What is AI? Answer in 100 words'
)
print(response.text)

## What is AI?

Artificial Intelligence (AI) broadly refers to the simulation of human intelligence processes by computer systems. This includes learning, reasoning, and self-correction. 

Essentially, AI aims to create machines that can perform tasks typically requiring human intelligence. It's achieved through techniques like machine learning (where systems learn from data) and deep learning (using artificial neural networks). 

AI powers things like voice assistants (Siri, Alexa), recommendation systems (Netflix, Spotify), and increasingly, complex applications in healthcare, finance, and transportation. It's not *one* thing, but a diverse field constantly evolving.






## 1. Define Tools

Tools are simply Python functions that the model can "call". We need to define these functions and provide clear docstrings so the model understands when and how to use them.

In [ ]:
def add(a: float, b: float) -> float:
    """Adds two numbers.

    Args:
        a: The first number.
        b: The second number.

    Returns:
        The sum of a and b.
    """
    print(f"[Tool defined in AI Summit] Calling add({a}, {b})")
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiplies two numbers.

    Args:
        a: The first number.
        b: The second number.

    Returns:
        The product of a and b.
    """
    print(f"[Tool defined in AI Summit] Calling multiply({a}, {b})")
    return a * b

def get_weather(city: str) -> str:
    """Get the current weather for a given city.

    Args:
        city: The name of the city.

    Returns:
        A string describing the weather.
    """
    print(f"[Tool defined in AI Summit] Calling get_weather('{city}')")
    # Mock data for demonstration
    weather_data = {
        "Bangalore": "Windy, 17°C",
        "Kolkata": "Windy, 28°C",
        "Chennai": "Sunny, 32°C",
        "Hyderabad": "Sunny, 30°C"
    }
    return weather_data.get(city, "Weather data not available for this city.")

import random
def get_stock_price(ticker: str) -> float:
    """Gets the current stock price for a given ticker symbol.

    Args:
        ticker: The stock ticker symbol (e.g., 'GOOG', 'AAPL', 'MSFT').

    Returns:
        A simulated current stock price for the given ticker.
    """
    print(f"[Tool defined in AI Summit] Calling get_stock_price('{ticker}')")
    # Simulate stock prices for demonstration purposes
    mock_prices = {
        "GOOG": 175.50,
        "AAPL": 215.25,
        "MSFT": 450.70,
        "AMZN": 180.90,
        "NVDA": 190.00
    }
    price = mock_prices.get(ticker.upper())
    if price:
        # Add some randomness to simulate fluctuations
        return round(price * (1 + (random.random() - 0.5) * 0.05), 2) # +/- 2.5% fluctuation
    else:
        return -1.0 # Indicate stock not found

def calculate_indian_income_tax(taxable_income: float, regime: str) -> float:
    """Calculates the income tax for an employee in India based on taxable income and tax regime.
    Args:
        taxable_income: The total taxable income.
        regime: The chosen tax regime ('new' or 'old').
    Returns:
        The calculated income tax.
    Raises:
        ValueError: If an invalid tax regime is provided.
    """
    print(f"[Tool] Calling calculate_indian_income_tax({taxable_income}, '{regime}')")
    tax = 0.0
    if regime == 'new':
        # Updated New Tax Regime slabs as provided by the user
        if taxable_income <= 1200000:
            tax = 0
        elif taxable_income <= 1600000:
            tax = (taxable_income - 1200000) * 0.15
        elif taxable_income <= 2000000:
            tax = (400000 * 0.15) + (taxable_income - 1600000) * 0.20
        elif taxable_income <= 2400000:
            tax = (400000 * 0.15) + (400000 * 0.20) + (taxable_income - 2000000) * 0.25
        else:
            tax = (400000 * 0.15) + (400000 * 0.20) + (400000 * 0.25) + (taxable_income - 2400000) * 0.30
    elif regime == 'old':
        if taxable_income <= 250000:
            tax = 0
        elif taxable_income <= 500000:
            tax = (taxable_income - 250000) * 0.05
        elif taxable_income <= 1000000:
            tax = 12500 + (taxable_income - 500000) * 0.20
        else:
            tax = 112500 + (taxable_income - 1000000) * 0.30
    else:
        raise ValueError("Invalid tax regime. Choose 'new' or 'old'.")

    return tax


## 2. View Current Available Models

We can use the ListModels to see the list of available models and their supported methods.

In [ ]:
for model in genai.list_models():
  print(model.name)

models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview

In [ ]:
import pprint
for model in genai.list_models():
  if 'generateContent' in model.supported_generation_methods:
    pprint.pprint(model)

Model(name='models/gemini-2.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 2.5 Flash',
      description=('Stable version of Gemini 2.5 Flash, our mid-size multimodal model that '
                   'supports up to 1 million tokens, released in June of 2025.'),
      input_token_limit=1048576,
      output_token_limit=65536,
      supported_generation_methods=['generateContent',
                                    'countTokens',
                                    'createCachedContent',
                                    'batchGenerateContent'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)
Model(name='models/gemini-2.5-pro',
      base_model_id='',
      version='2.5',
      display_name='Gemini 2.5 Pro',
      description='Stable release (June 17th, 2025) of Gemini 2.5 Pro',
      input_token_limit=1048576,
      output_token_limit=65536,
      supported_generation_methods=['generateContent',
                  

### 3. Initialize the Model - Prompt Version

In [ ]:
# Create a chat session
# Initialize Model with Generative AI
model = genai.GenerativeModel(
    model_name='gemma-3-27b-it'
)
# Start a chat session with the model initialized above
chat = model.start_chat()

In [ ]:
import json
def postProcessResponse(response):
  # Load the JSON Object String using loads()
  payload = json.loads(response.text)
  # Extract the function_name from the JSON Parsed Object
  fn_name = payload['function_name']
  # Extract the arguments from the JSON Parsed Object
  args    = payload['arguments']
  # Get the function object from the defined global symbols in the notebook.
  fn      = globals()[fn_name]
  # Call the function with the arguments passed
  result  = fn(*args)
  #get_stock_price
  return result

def getOutput(task):
  # Define the Prompt Here that will let the LLM output the required tool to call
  prompt  = '''
  You are a function router. The following are the functions you have in your disposal: -
  1) add(a, b): Returns the addition of a and b
  2) multiply(a, b): Returns the multiplication of a and b
  3) get_weather(city): Returns the current weather for the given city
  4) get_stock_price(ticker): Returns the current stock price for the given ticker symbol
  5) calculate_indian_income_tax(taxable_income, regime): Calculates the income tax for an employee in India based on taxable income depending on the regime
    which can be either 'new' or 'old'.
  Given a user request, decide which function to call and its arguments.
  Return the result as a single valid JSON object with this exact schema:
  {
    "function_name": "<string name of the function to call>",
    "arguments": [<list of arguments as JSON values>]
  }
  Requirements:
  - Output ONLY the JSON object, with no extra text.
  - Keys must be: "function_name" and "arguments".
  - "arguments" must be a JSON array.
  - Do NOT wrap the JSON in backticks or markdown.
  Example:
  User request: "Get the price of Google stock"
  Output:
  {"function_name": "get_stock_price", "arguments": ["GOOG"]}
  Now process this request: ''' + task
  # Use the send_message() method of the chat session object to send the prompt
  response = chat.send_message(prompt)
  # Return the output
  return response

In [ ]:
response = getOutput("What is the current stock price of NVIDIA(NVDA)?")
print(response.text)
print(postProcessResponse(response))

{"function_name": "get_stock_price", "arguments": ["NVDA"]}
[Tool defined in AI Summit] Calling get_stock_price('NVDA')
185.96


In [ ]:
response = getOutput("What is the weather like at Bangalore?")
print(response.text)
print(postProcessResponse(response))

{"function_name": "get_weather", "arguments": ["Bangalore"]}
[Tool defined in AI Summit] Calling get_weather('Bangalore')
Windy, 17°C


In [ ]:
response = getOutput("Calculate the Indian income tax for a taxable income of 1800000 under the new regime.")
print(response.text)
print(postProcessResponse(response))

{"function_name": "calculate_indian_income_tax", "arguments": [1800000, "new"]}
[Tool] Calling calculate_indian_income_tax(1800000, 'new')
100000.0


## 4. Easier Alternative: Pass the Function Names as a list and let the LLM call the appropriate function 😀

Now we can send messages to the chat session. If the model decides it needs to use a tool to answer, it will automatically call the Python function and use the result.

In [ ]:
# Add the tools to a list
tools_list = [add, multiply, get_weather, get_stock_price, calculate_indian_income_tax]
# Initialize Model with Generative AI
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash-lite',
    tools = tools_list
)
# Start a chat session with automatic function calling enabled
chat = model.start_chat(enable_automatic_function_calling=True)

NameError: name 'add' is not defined

In [ ]:
response = chat.send_message("What is the current stock price of Google (GOOG)?")
print(response.text)

NameError: name 'chat' is not defined

In [ ]:
response = chat.send_message("What is 57 multiplied by 3?")
print(response.text)

NameError: name 'chat' is not defined

In [ ]:
response = chat.send_message("What is the weather like in Bangalore today?")
print(response.text)

[Tool defined in AI Summit] Calling get_weather('Bangalore')
The weather in Bangalore is windy and 17°C.


In [ ]:
response = chat.send_message("If I buy 2 apples at $1.5 each and 3 oranges at $2 each, how much is the total?")
print(response.text)

[Tool defined in AI Summit] Calling multiply(1.5, 2.0)
[Tool defined in AI Summit] Calling multiply(2.0, 3.0)
[Tool defined in AI Summit] Calling add(3.0, 6.0)
The total is $9.


In [ ]:
# Add the tools to a list
tools_list = [add, multiply, get_weather, get_stock_price, calculate_indian_income_tax]
# Initialize Model with Generative AI
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools = tools_list
)
# Start a chat session with automatic function calling enabled
chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
response = chat.send_message("Calculate the Indian income tax for a taxable income of 1800000 under the new regime.")
print(response.text)

[Tool] Calling calculate_indian_income_tax(1800000.0, 'new')
The calculated income tax for a taxable income of 18,00,000 under the new regime is 1,00,000.


## 5. Exploring the Conversation History

You can inspect the chat history to see how the model called the functions and how the results were fed back.

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(f"Role: {content.role}")
    if part.function_call:
        print(f"Function Call: {part.function_call.name}{dict(part.function_call.args)}")
    elif part.function_response:
         print(f"Function Response: {part.function_response.response}")
    else:
        print(f"Text: {part.text}")
    print("-" * 20)

Role: user
Text: Calculate the Indian income tax for a taxable income of 1800000 under the new regime.
--------------------
Role: model
Function Call: calculate_indian_income_tax{'taxable_income': 1800000.0, 'regime': 'new'}
--------------------
Role: user
Function Response: <proto.marshal.collections.maps.MapComposite object at 0x7fd418f61b80>
--------------------
Role: model
Text: The calculated income tax for a taxable income of 18,00,000 under the new regime is 1,00,000.
--------------------


Another example to illustrate Plain Vanilla LLM

In [ ]:
# Let's say we are planning to ask the agent about a trip itinerary to Rohtang Pass next weekend.
response = chat.send_message("Can you build me a 1-Day itinerary for Rohtang Pass for next weekend?")
print(response.text)

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2000.05ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1973.21ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2053.96ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 7111.92ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 3671.85ms


I am sorry, I cannot help you with that. I can only answer questions that can be answered using the available tools. I cannot create a travel itinerary.


Even though the model gave some hint that these months are not suitable for visiting Rohtang Pass as it is closed due to snow, we want the LLM to think and check if Rohtang Pass is closed. If closed, provide an itinerary of a nearby destination that is open instead.
What if the agent can first check the status from Google using an API whether Rohtang Pass is open or not and if it is open then only give us the itinerary, otherwise it suggests nearby places and what we can do.
This workflow to the agents is provided using LangChain and LangGraph

Watch out for the session by Chinmay and others after lunch 😀

# Real-Life Example - AI Driven Student Email Automation Script from csv content

# Automated Attendance Monitoring & Email Generation Using Gemini API

This notebook demonstrates an end-to-end workflow for automating attendance monitoring and generating personalized email alerts for students falling below a required attendance threshold. The workflow:

    - Accepts CSV content pasted directly in the script or input
    - Accepts a natural-language instruction
    - Uses OpenAI API to extract:
        * Required action
        * Attendance condition
        * Email body
    - Filters students based on attendance threshold
    - Sends emails to parents

In [ ]:
!pip install google-genai # This installs the Google Generative AI client library.
# After installation, you'll typically use genai.configure(api_key=YOUR_API_KEY) to authenticate your access to the Gemini API.

In [ ]:
import google.generativeai as genai
import json

In [ ]:
""" An API (Application Programming Interface) is an encrypted string that identifies a
    Google Cloud project for quota, billing and monitoring purposes."""

# GEMINI_API_KEY=""

' An API (Application Programming Interface) is an encrypted string that identifies a\n    Google Cloud project for quota, billing and monitoring purposes.'

In [ ]:
# Configures the client by authenticating it to use google.generativeai library within the session.
genai.configure(api_key=GEMINI_API_KEY)

NameError: name 'GEMINI_API_KEY' is not defined

In [ ]:
system_prompt= """From the provided dataset, identify all the students whose attendance is below 75%. For each of these students,
                    provide the following details in the specified format:
                    - Student Name: {Student Name}
                    - Roll No.: {Student Number}
                    - Parent Email: {Parent Email}m
                    - Proctor Email: {Proctor Email}
                    - Email Body: {Email Body}

                    Compose a professional email adressed to parents or gaurdians of these students. the email should:
                    - Begin with a polite greeting and clearly state the purpose of the email.
                    - Inform the parents that their ward is falling behind due to attendance.
                    - Emphasize the importance of improving attednance immediately to ensure academic success.
                    - Be polite yet firm, encouraging parents to actively suport their child's regular attendance.
                    - Include the proctor's name and email in the maessage, encouraging the parents to reach out to the
                      proctor to schedule an in-person meeting at their earliest mutual convenience.
                    - Convey urgency and concern while remaining respectful and professional.
                    - End with a call to action, prompting parents to take immediate steps to address the issue.

                    Use the following placeholders in the email body:
                    - {Student Name} for the student's name.
                    - {Roll Number} for the student's roll number.
                    - {Attendance Percentage} for the student's attendance percentage.
                    - {Proctor's Name} for the proctor's name.
                    - {Proctor's email} for the proctor's email.
                """

In [ ]:
for m in genai.list_models():
    print(m.name)

models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview

In [ ]:
import google.generativeai as genai

# This code initializes a generative AI model from the Google Generative AI library.
model = genai.GenerativeModel(
    # We will use a free version like Gemini 2.5 Flash.
    # It is suitable for tasks such as answering questions and summarizing text.
    model_name="models/gemini-2.5-flash",
    # The 'system_instruction' argument provides a system prompt to guide the model's behavior.
    # This prompt sets the context and instructs the model on how to process the input
    # and format its output, e.g., to extract student data and compose emails as defined in 'system_prompt'.
    system_instruction=system_prompt
)

In [ ]:
# This variable, `csv_content`, holds our student dataset.
# For this demonstration, we're embedding the CSV data directly as a multi-line string
# instead of loading it from a file. This simplifies the setup for our class example.
# Each line represents a student record, and the first line is the header defining the columns.
csv_content = """Student Roll Number,Student Name,Gender,CGPA,Date of Birth,Courses,Marks in Each Course,Attendance Percentage,Proctor Name,Proctor Email,Parent Email
CSAI001,Rohan Sharma,M,8.2,2003-02-11,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","82;78;85;80;75",88,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI002,Ananya Gupta,F,9.1,2003-07-29,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","90;92;89;94;88",95,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI003,Arvind Menon,M,7.5,2002-12-09,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","70;72;68;75;65",72,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI004,Priya Iyer,F,8.8,2003-04-14,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","88;85;90;87;89",97,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI005,Vikram Reddy,M,6.9,2002-11-30,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","60;58;62;55;57",68,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI006,Neha Kulkarni,F,9.3,2003-06-18,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","92;94;90;95;91",99,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI007,Aditya Nair,M,7.8,2003-01-25,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","74;77;79;72;70",82,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI008,Sana Khan,F,8.5,2002-10-17,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","86;82;84;88;83",90,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI009,Harshit Verma,M,7.2,2003-05-05,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","68;65;70;72;66",74,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI010,Divya Suresh,F,8.9,2003-03-09,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","90;87;91;89;88",96,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI011,Karthik Raman,M,8.1,2003-02-22,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","80;82;81;83;79",89,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI012,Aishwarya Patil,F,7.9,2002-09-19,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","75;78;74;72;71",85,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI013,Manoj Sen,M,6.5,2003-07-02,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","55;58;60;52;50",70,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI014,Simran Kaur,F,9.0,2003-01-11,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","91;93;90;92;89",98,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI015,Rahul Chatterjee,M,8.4,2002-08-28,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","83;80;85;82;81",87,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI016,Nisha Mukherjee,F,7.1,2002-06-16,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","65;67;69;70;63",73,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI017,Ritesh Jain,M,8.6,2003-03-03,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","88;84;86;90;85",92,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI018,Swati Deshmukh,F,7.4,2002-12-21,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","72;74;70;68;69",76,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI019,Arjun Pillai,M,9.2,2003-10-10,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","94;91;92;93;95",99,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI020,Isha Rathod,F,8.0,2003-05-14,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","82;80;78;79;77",84,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI021,Tejas Gowda,M,7.6,2003-06-07,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","70;72;75;73;68",71,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI022,Ragini Singh,F,9.4,2003-02-03,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","95;94;96;93;92",100,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI023,Shivam Malhotra,M,6.8,2002-09-01,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","58;55;60;62;57",69,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI024,Harini Sekar,F,8.7,2003-11-19,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","87;89;88;90;85",94,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI025,Rehan Qureshi,M,7.0,2003-01-15,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","65;63;68;70;66",72,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI026,Anjali Dutta,F,8.3,2003-03-27,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","84;86;83;82;80",91,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI027,Kunal Thakur,M,7.3,2002-10-05,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","69;72;71;70;68",74,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI028,Mansi Reddy,F,9.0,2003-07-23,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","92;90;93;91;89",97,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI029,Amit Dubey,M,6.7,2002-08-12,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","58;60;55;57;59",65,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI030,Pooja Bansal,F,8.2,2003-04-06,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","82;84;80;83;81",89,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI031,Nitin Arora,M,7.9,2003-09-14,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","78;76;79;77;75",77,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI032,Sonali Jadhav,F,8.6,2003-08-30,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","88;86;87;89;85",93,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI033,Yash Mittal,M,6.9,2002-12-05,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","62;60;64;61;63",69,Dr. S. Krishnan,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI034,Keerthi Menon,F,9.1,2003-11-27,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","93;94;92;90;91",98,Dr. Anita Rao,arupdas.research.iitm@gmail.com,das12997@gmail.com
CSAI035,Adarsh Jena,M,7.4,2002-07-16,"Machine Learning;Deep Learning;Natural Language Processing;Computer Vision;Reinforcement Learning","70;72;68;69;71",73,Dr. Meenakshi Ray,arupdas.research.iitm@gmail.com,das12997@gmail.com
"""

In [ ]:
# This `user_prompt` variable is crucial because it's the specific instruction we give to our Generative AI model.
# It combines two main parts:
# 1.  The `csv_content`: This is the raw student data we defined earlier, which the AI needs to process.
# 2.  The Instruction: This tells the AI *what to do* with the data.
#
# In this case, the instruction is to:
#   -  Find all students whose 'Attendance Percentage' is below 75%.
#   -  Format the output as a JSON object, specifically with an `action` of "send_email",
#      an `attendance_threshold` of 75, an `email_body` (which the AI will generate based on the system prompt),
#      and a list of `students` who meet the criteria.
#
# This structured approach helps ensure the AI understands exactly what information to extract
# and how to present it, making it easier for our Python code to parse and use the AI's output.
user_prompt = f"""Below is student dataset in CSV format.

                  {csv_content}

                  Your task:
                  Extract all students with attendance < 75% and produce structured JSON:
                  {{
                      "action": "send_email",
                      "attendance_threshold": 75,
                      "email_body": "...",
                      "students": [ ... ]
                  }}
              """

In [ ]:
# This line sends the 'user_prompt' (which contains the CSV data and instructions) to the
# generative AI model (initialized as 'model'). The model processes this input and
# generates a response. The entire response object is stored in the 'response' variable.
response = model.generate_content(user_prompt)

In [ ]:
print(response)

In [ ]:
# This line extracts the actual text content from the 'response' object generated by the model.
# The '.text' attribute accesses the string output, and '.strip()' removes any leading or
# trailing whitespace (like newlines or spaces) to get a clean string, which is then
# stored in the 'raw_text' variable.
raw_text = response.text.strip()

In [ ]:
print(raw_text)

NameError: name 'raw_text' is not defined

In [ ]:
# This block of code is designed to clean the 'raw_text' received from the Generative AI model.
# The model sometimes wraps its JSON output in markdown code block syntax (e.g., ```json...```).
# These wrappers need to be removed before the text can be parsed as a valid JSON object.

# Checks if the 'raw_text' string starts with '```'. This indicates the presence of a markdown code block.
if raw_text.startswith("```"):
    # If it starts with '```', this line removes all leading and trailing backticks.
    raw_text = raw_text.strip("`")
    # This line specifically removes the 'json' keyword that often follows the opening backticks
    # (e.g., ```json). It only replaces the first occurrence to avoid removing 'json' from actual content.
    raw_text = raw_text.replace("json", "", 1).strip()

In [ ]:
print(raw_text)

{"function_name": "calculate_indian_income_tax", "arguments": [1800000, "new"]}


In [ ]:
try:
    parsed = json.loads(raw_text)
    print(parsed)
except Exception as e:
    print("[ERROR] Failed to parse JSON:", e)
    print("Raw LLM response:", raw_text)
    raise

{'function_name': 'calculate_indian_income_tax', 'arguments': [1800000, 'new']}


In [ ]:
try:
    parsed = json.loads(raw_text)
    print(parsed)
except Exception as e:
    print("[ERROR] Failed to parse JSON:", e)
    print("Raw LLM response:", raw_text)
    raise

{'function_name': 'calculate_indian_income_tax', 'arguments': [1800000, 'new']}
